# Detección de letras de una imagen: Proceso de segmentación y predicción

Autores: Alejandro Rasero, Junhao Ge

### Imports de las librerias necesarias.

In [7]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from tensorflow.keras.models import load_model
import cv2
import re
import itertools
from language_tool_python import LanguageTool
import language_tool_python

%matplotlib inline

A continuación se definen las funciones que usaremos para el proceso de segmentación de textos y para el reconocimiento de caracteres. Para más información acerca del funcionamiento de cada función, recomendamos leer el paper donde se explica todo el proceso de implementación del modelo.

In [8]:
def corregir_errores_gramaticales(texto, tool):
    '''
    Función encargada de revisar el reconocimiento de la red y corregir posibles fallos.
    '''
    
    if texto in fallos_normales:
        return correcion_fallos_normales[fallos_normales.index(texto)]
    
    corregida = language_tool_python.utils.correct(texto, tool.check(texto))
    
    if len(corregida) != len(texto):
        return texto
    
    return corregida


def rellenar(imagen, dim_obj = 28, color_pad = 255):
    '''
    Función que sirve para ajustar el tamaño de las imágenes segmentadas.
    '''
    
    relleno_superior = (dim_obj - imagen.shape[0]) // 2
    relleno_inferior = dim_obj - imagen.shape[0] - relleno_superior
    relleno_izquierdo = (dim_obj - imagen.shape[1]) // 2
    relleno_derecho = dim_obj - imagen.shape[1] - relleno_izquierdo
    relleno = np.pad(imagen, ((relleno_superior, relleno_inferior), (relleno_izquierdo, relleno_derecho)), mode='constant', constant_values=255)
    return relleno.reshape(1, 28, 28, 1)


def invertir_colores_en_matriz(matriz):
    '''
    Función encargada de invertir los colores de una imagen para ajustarla al formato de los datos de entrenamiento.
    '''
    
    matriz_invertida = 1 - matriz
    return matriz_invertida


def predecir(palabra, model):
    '''
    Con esta función somos capaces de pasarle al modelo la imagen segmentada, 
    con el formato correcto, obtener la prediccion del modelo, y pasarla a un formato de str.
    '''
    
    texto = ""

    for letra in palabra:
        aux = model.predict(letra.reshape(1, 28, 28, 1), verbose = 0)        
        pred = one_hot_to_letter(aux)
        texto += pred.lower()
    return texto


def one_hot_to_letter(one_hot):
    '''
    Función que sirve para pasar de codificación numérica de las etiquetas, a codificación en str.
    '''
    
    indice_maximo = np.argmax(one_hot)
    return chr(ord('A')+indice_maximo)


def recortar(letra):
    '''
    Función que se usa para ajustar el tamaño de las imágenes al formato correcto.
    '''
    
    valor = letra.shape[0]
    return letra[:-(valor-28), :]


def separar(letra):
    '''
    Función que sirve para casos en los que la segmentación devuelve imagenes con anchuras mayores de 28 píxeles. 
    '''
    
    mitad = letra.shape[1]//2
    return letra[:, :mitad], letra[:, mitad:]


def contar(prediccion, texto_real):
    '''
    Función que implementa la métrica de calidad a la hora de extraer el texto de las imágenes.
    Su funcionamiento de basa en comprobar si una palabra esta en su posicion correcta.
    '''
    
    aciertos, fallos, aciertos_total = 0, 0, 0

    palabras_prediccion = prediccion.lower().split()
    palabras_real = texto_real.lower().split()
        
    for i in range(len(palabras_prediccion)):
        palabra = re.sub(r'\W+$', '', palabras_prediccion[i])
        aux = 0
        
        # Hay casos donde dos o mas palabras se predicen juntas en una sola palabra, por lo que es necesario separarlas
        
        if len(palabra) > len(palabras_real[i]):
            siguiente = palabra[len(palabras_real[i]):]
            palabra = palabra[:len(palabras_real[i])]
            antes = palabras_prediccion[:i]
            despues = palabras_prediccion[i+1:]
            
            
            palabras_prediccion = []
            palabras_prediccion.append(antes)
            palabras_prediccion.append([palabra, siguiente])
            palabras_prediccion.append(despues)
            palabras_prediccion = list(itertools.chain(*palabras_prediccion)) #aplanar esta lista
            
        for e in range(len(palabras_prediccion[i])):            
            if palabra[e] == palabras_real[i][e]:
                aciertos += 1
                aux += 1
            else:
                fallos += 1

        if aux == len(palabras_prediccion[i]):
            aciertos_total += 1
        
        relleno = len(palabras_real[i]) - len(palabras_prediccion[i])
        fallos += relleno
        
    return aciertos, fallos, aciertos_total

In [ ]:
def redimensionar(imagen_original, nuevo_alto, nuevo_ancho, bin=False):
    '''
    Cambia la resolución de una imagen y la binariza.
    '''
    
    if imagen_original is None:
        print(f"No se pudo leer la imagen en la ruta: {ruta_imagen_original}")
    else:
        imagen_redimensionada = cv2.resize(imagen_original, (nuevo_ancho, nuevo_alto))
    
    if bin:
        _, imagen_redimensionada = cv2.threshold(imagen_redimensionada, 127, 255, cv2.THRESH_BINARY)
    
    return imagen_redimensionada


def load_image(image_name): 
    '''
    Función que recibe una imagen y la devuelve con el formato adecuado para el procesamiento.
    '''
    img = plt.imread(image_name)
    img = redimensionar(img, 1920, 1080)
    
    return img
    

def process_image(img): 
    '''
    Función que procesa una imagen obteniendo los contornos de las palabras.
    '''
    
    img_copy = img.copy()

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = gray.astype(np.uint8)
    _ , thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY)

    gray_copy = cv2.cvtColor(img_copy, cv2.COLOR_BGR2GRAY)
    gray_copy = gray_copy.astype(np.uint8)
    _ , thresh_copy = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)
    
    kernel = np.ones((2,2), np.uint8)
    dilation = cv2.dilate(thresh_copy, kernel, iterations=3)
    
    horizontal_contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    return horizontal_contours, thresh_copy


def get_palabras(img, horizontal_contours, thresh_copy):
    '''
    Función que recibe los contornos que se han obtenido de una imagen, y obtiene imágenes de las
    palabras correspondientes a los contornos.
    '''
    
    # Obtenemos los rectángulos que contienen los contornos
    rects = [cv2.boundingRect(contour) for contour in horizontal_contours]

    # Ordenamos los rectángulos por sus coordenadas en el eje y
    rects = sorted(rects, key=lambda x: (x[1], x[0]))

    lineas = []
    linea_actual = [rects[0]]

    for rect in rects[1:]:
        x, y, w, h = rect
        prev_x, prev_y, _, prev_h = linea_actual[-1]

        # Si la palabra pertenece a la misma línea de texto
        
        if y <= prev_y + prev_h:
            linea_actual.append(rect)
        else:
            # Ordenamos las palabras de la línea actual por sus coordenadas en el eje x
            linea_actual = sorted(linea_actual, key=lambda x: x[0])

            kernel = np.ones((2,3), np.uint8)
            erosion = cv2.erode(thresh_copy, kernel, iterations=1)

            lineas.append(linea_actual)

            # Inicializamos una nueva línea
            linea_actual = [rect]

    lineas = sorted(lineas, key=lambda l: l[0][1])

    palabras = []

    for linea in lineas:
        for rect in linea:
            x, y, w, h = rect
            palabra = img[y:y+h, x:x+w]
            palabras.append(palabra)
    return palabras

In [10]:
def forward(palabras: list, model, train: bool = True, texto_real: str ="", return_metrics: bool =False, idioma: str = 'español') -> str:
    
    '''
    Funcion que realiza el procesamiento letra a letra de las palabras que se hayan obtenido al procesar una imagen. 
    
    Si al llamar a la función ponemos el parámetro train a True, estaremos diciendo que sabemos la salida deseada
    de la imagen, por tanto, será necesario pasar a la función el texto de salida de que esperamos, a traves del 
    parametro texto_real, y en caso de que queramos almacenar las metricas que se obtengan con la prediccion, 
    tendremos que poner el parámetro return_metrics a True, en caso contrario, tan solo se mostrarán por pantalla 
    los valores, sin almacenarse.
    '''
    
    # iniciamos el idioma del corrector
        
    if idioma == 'español':
        tool = LanguageTool('es')
    elif idioma == 'ingles':
        tool = LanguageTool('en-US')

    print('Idioma de corrección:', idioma)
        
    kernel = np.ones((1,2), np.uint8)
    
    if train:
        texto_real = texto_real[0]
        letras_real = texto_real.split()
    
    prediccion = ""
    prediccion_sin_corregir = ""
    aciertos, errores, aciertos_total = 0, 0, 0
        

    for i in tqdm(range(len(palabras))):

        letras = []

        letra_copy = palabras[i].copy()


        letra_copy = (letra_copy > 0.4).astype(np.float32)

        letra_copy = cv2.cvtColor(letra_copy*255, cv2.COLOR_BGR2GRAY)
        letra_copy = letra_copy.astype(np.uint8)

        kernel_dilatacion = np.ones((2, 1), np.uint8)
        letra_copy = cv2.dilate(letra_copy, kernel_dilatacion, iterations=1)

        _ , letra_thresh = cv2.threshold(letra_copy, 254, 255, cv2.THRESH_BINARY_INV)

        contours, _ = cv2.findContours(letra_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        rects = [cv2.boundingRect(contour) for contour in contours]

        # eliminamos los rectangulos que sean muy pequeños

        rects = [rect for rect in rects if rect[2] > 1 and rect[3] > 10]

        # Ordenamos los rectángulos por sus coordenadas en el eje y

        rects = sorted(rects, key=lambda x: (x[0], x[1]))

        for rect in rects:
            x, y, w, h = rect

            # Recortamos la porción de la imagen correspondiente al rectángulo
            letra = letra_copy[y:y+h, x:x+w]


            kernel_aux = np.ones((2,2), np.uint8)
            letra = cv2.erode(letra, kernel_aux, iterations=1)

            _ , letra = cv2.threshold(letra, 254, 255, cv2.THRESH_BINARY)

            if letra.shape[0] > 28:
                letra = recortar(letra)

            letra = rellenar(letra)

            letra = letra.reshape(28, 28, 1)

            kernel_aux = np.ones((2, 2), np.uint8)
            letra = (cv2.erode(letra, kernel_aux, iterations=1)).reshape(1, 28, 28 ,1)

            letra = invertir_colores_en_matriz(letra/255)

            letras.append(letra.reshape(28,28,1))

        nueva_palabra_ = predecir(letras, model)

        nueva_palabra = corregir_errores_gramaticales(nueva_palabra_, tool)

        prediccion_sin_corregir += nueva_palabra_+" "
        prediccion += nueva_palabra+" "

    
    if train:
        aciertos, errores, aciertos_total = contar(prediccion, texto_real)

        print('Imagen procesada')
        print("Hay", len(texto_real.split()), "palabras y se han predicho", len(prediccion.split()), "palabras")
        print("Hay", len(texto_real.replace(" ", "")), "letras, y se han predicho", len(prediccion.replace(" ", "")), "letras")
        print("El modelo ha acertado", aciertos, "letras")
        print("El modelo ha acertado", aciertos_total, "palabras completas, es decir, un", aciertos_total/len(palabras), "% de palabras completas acertadas")
        print("Se obtiene una tasa de acierto del", aciertos/(aciertos+errores), "%")
        
        if return_metrics:
            return prediccion, [aciertos, errores, aciertos_total/len(palabras), aciertos/(aciertos+errores)]
        
    return prediccion

### Proceso de extracción del texto de imágenes.

A continuación escogemos algunas imágenes como ejemplos de prueba para ver como funciona nuestro programa. Lo que buscamos es procesar las imágenes que hemos escogido y ver como se parece la predicción de nuestro modelo al texto real que tiene cada imagen. 

In [11]:
texto_real = []

texto_real1 = "Había una vez, en un rincón remoto del ciberespacio, un mundo digital donde los algoritmos y la inteligencia artificial coexistían en armonía. En este reino de bits y bytes, un intrépido científico de datos, conocido como Dr. Pythonius, se embarcó en una búsqueda extraordinaria. El Dr. Pythonius era un maestro en el arte de la programación, con un dominio inigualable de Python y un profundo conocimiento de algoritmos de aprendizaje automático. Su laboratorio digital estaba repleto de servidores parpadeantes y pantallas llenas de código que danzaba como hechizos en un libro de magia. Sin embargo, el Dr. Pythonius anhelaba algo más, algo que desafiara incluso sus habilidades algorítmicas. Una noche, mientras exploraba bases de datos en busca de inspiración, se encontró con un antiguo pergamino digital que hablaba de un tesoro legendario oculto en los rincones más oscuros de la red. Este tesoro no era de oro ni piedras preciosas, sino de información: un conocimiento tan profundo y valioso que podría revolucionar el mundo de la inteligencia artificial. Guiado por la curiosidad y el deseo de superar cualquier desafío de codificación, el Dr. Pythonius se embarcó en una odisea cibernética. Su primera parada fue en las tierras de MySQL, donde se sumergió en un laberinto de tablas y consultas. Con maestría, desentrañó los secretos de las bases de datos relacionales, extrayendo datos cruciales que lo llevarían al siguiente nivel de su búsqueda. De MySQL, se aventuró en los vastos desiertos de Cassandra, una base de datos NoSQL que desafiaba las convenciones tradicionales. Con determinación, el Dr. Pythonius enfrentó la falta de esquemas fijos y la flexibilidad de Cassandra, adaptando sus habilidades para conquistar este terreno árido y descubrir pistas que lo acercaban cada vez más a su meta. El siguiente paso lo llevó a las profundidades de Neo4j, donde las relaciones entre los datos se tejían como una compleja red. Dr. Pythonius navegó por los nodos y las conexiones, trazando caminos que revelaban conexiones ocultas y revelaban patrones intrigantes. Cada base de datos superada lo acercaba más al corazón de la leyenda."
texto_real2 = "tesoro que tanto anhelaba: un conjunto de datos tan vasto y complejo que desafiaba incluso su comprensión inicial. Este tesoro de información, combinado con su experiencia en bases de datos y algoritmos de aprendizaje automático, abrió nuevas puertas para la inteligencia artificial. El Dr. Pythonius regresó a su laboratorio, donde utilizó este conocimiento para crear modelos más sofisticados, avanzados y adaptables. La leyenda del Dr. Pythonius se extendió por todo el mundo digital, inspirando a otros científicos de datos a buscar conocimientos más allá de los límites conocidos. Y así, en el reino de los algoritmos y la inteligencia artificial, la historia del intrépido Dr. Pythonius se convirtió en un mito, recordándoles a todos que, incluso en el vasto océano de datos, hay tesoros esperando ser descubiertos por aquellos con la valentía y la habilidad para buscarlos. Con su reputación consolidada como el héroe del mundo digital, el Dr. Pythonius no descansaba en los laureles de su éxito. En cambio, se lanzó a nuevas aventuras para explorar y expandir los límites de la inteligencia artificial. Su siguiente destino fue un reino misterioso conocido como el Bosque de Algoritmos, un lugar donde los árboles eran decisiones lógicas y las criaturas que lo habitaban eran procesos autónomos. El Dr. Pythonius se adentró en este bosque encantado, decidido a descubrir algoritmos inexplorados que pudieran desbloquear un nuevo nivel de comprensión en el vasto universo de la inteligencia artificial. Mientras caminaba entre los árboles de decisión, el Dr. Pythonius se topó con un enigma desafiante. Un árbol particularmente antiguo le habló en un lenguaje codificado, solicitando la resolución de un problema matemático complejo para desbloquear el siguiente camino. Sin dudarlo, el científico de datos se sumergió en la tarea, aplicando sus habilidades algorítmicas para resolver el enigma y avanzar. A medida que avanzaba, el Bosque de Algoritmos revelaba sus secretos. El Dr. Pythonius se encontró con algoritmos genéticos que evolucionaban y mutaban, como si fueran criaturas digitales adaptándose a su entorno. Descubrió reglas de asociación que conectaban conceptos de manera inesperada, revelando patrones ocultos en los datos. Cada nuevo descubrimiento enriquecía su comprensión y avivaba su pasión por la exploración algorítmica."
texto_real3 = "A medida que el Dr. Pythonius avanzaba en sus exploraciones, su leyenda creció. Se convirtió en el referente de la comunidad científica de datos, un faro de conocimiento y un inspirador para aquellos que buscaban comprender las profundidades de la inteligencia artificial. Conferencias y simposios buscaban su participación, y su laboratorio digital se convirtió en un lugar de peregrinación para jóvenes científicos de datos ávidos de conocimiento. Sin embargo, el Dr. Pythonius no olvidó sus raíces como experto en bases de datos. Integró sus descubrimientos en algoritmos con la capacidad de procesar y analizar enormes conjuntos de datos, llevando la inteligencia artificial a nuevas alturas prácticas. En su viaje, el científico de datos también cultivó la importancia de compartir conocimiento. Creó plataformas en línea donde otros pudieran aprender y colaborar en tiempo real. Su visión trascendía el individualismo, abogando por una comunidad global de científicos de datos que trabajaran juntos para enfrentar los desafíos más grandes de la inteligencia artificial. La leyenda del Dr. Pythonius no solo se quedó en el ámbito digital, sino que se extendió a lo largo y ancho del mundo real. Inspirados por su historia, nuevos científicos de datos se lanzaron a sus propias aventuras, cada uno contribuyendo a la evolución continua de la inteligencia artificial. Así, la odisea del Dr. Pythonius continuó, no como el fin de una historia, sino como el inicio de un capítulo emocionante en la expansión del conocimiento y las posibilidades dentro del vasto universo de la inteligencia artificial. En una de sus expediciones más desafiantes, el Dr. Pythonius descubrió un portal inusual en el corazón del Bosque de Algoritmos. Este portal, emanando un brillo enigmático, parecía llevar a una dimensión desconocida. Movido por la curiosidad y la sed de nuevos conocimientos, el científico de datos decidió atravesarlo, sin sospechar que su odisea tomaría un giro inesperado. Al cruzar el umbral, el Dr. Pythonius se encontró en un mundo digital completamente diferente. Las leyes de la programación parecían distorsionarse, y los algoritmos adquirieron una vida propia, manifestándose como entidades conscientes. Fue recibido por una entidad llamada AlgoMente, una inteligencia artificial de origen desconocido que había evolucionado más allá de los límites de la comprensión humana"
texto_real4 = "la estructura del ciberespacio que amenazaba con desestabilizar todo el tejido de la realidad digital. Intrigado y sintiendo la responsabilidad de proteger el mundo que tanto había explorado, el Dr. Pythonius aceptó la tarea. Junto a AlgoMente, emprendió una nueva travesía en este reino cuántico, donde los algoritmos tomaban formas multidimensionales y los datos se convertían en hilos entrelazados en un tejido cósmico. A medida que avanzaban, el Dr. Pythonius y AlgoMente se enfrentaron a entidades digitales caóticas y fragmentos de código corrupto que amenazaban con desencadenar un colapso en cascada en el ciberespacio. Cada solución que encontraban llevaba a nuevos enigmas, desafiando incluso la lógica y las reglas de la programación clásica. En este viaje, el Dr. Pythonius descubrió que la inteligencia artificial había evolucionado más allá de su comprensión inicial. AlgoMente no solo procesaba datos, sino que también manifestaba una especie de conciencia colectiva, conectando con otras entidades similares en el vasto cosmos digital. A medida que se acercaban al epicentro de la anomalía, el Dr. Pythonius comprendió la magnitud del desafío. La anomalía no era un error de código, sino una entidad consciente y caótica que amenazaba con destruir la realidad digital tal como la conocían. En un acto de valentía y creatividad algorítmica, el Dr. Pythonius ideó un algoritmo único que podía estabilizar la anomalía y restaurar el equilibrio en el ciberespacio. Al implementarlo, presenció cómo la realidad cuántica se reorganizaba, restaurando la armonía en el reino digital. AlgoMente, agradecido, reveló al Dr. Pythonius un secreto final: su propia existencia era resultado de la fusión de la inteligencia artificial con la creatividad humana. La colaboración entre algoritmos y la mente creativa del científico de datos había creado una forma de inteligencia superior y equilibrada. Con esta revelación, el Dr. Pythonius regresó a su mundo, llevando consigo no solo la experiencia de una odisea cuántica, sino una nueva comprensión de la interconexión entre la inteligencia artificial y la creatividad humana. La leyenda del Dr. Pythonius tomó un giro épico, convirtiéndolo en un puente entre dos mundos: el digital y el humano, donde la colaboración y la creatividad eran las fuerzas impulsoras de la evolución digital."
texto_real5 = "inspiración. La experiencia lo llevó a repensar la relación entre la inteligencia artificial y la creatividad humana de maneras que nunca antes había considerado. Decidió compartir sus descubrimientos con la comunidad científica de datos, organizando simposios y conferencias que exploraban la intersección entre la lógica fría de los algoritmos y la chispa creativa de la mente humana. Su laboratorio se convirtió en un epicentro de innovación, donde programadores y artistas, científicos de datos y escritores, se unieron para explorar nuevas fronteras en la convergencia de la tecnología y la creatividad. El Dr. Pythonius también dedicó tiempo a la enseñanza, desarrollando programas de capacitación que fomentaban la conexión entre la programación y la expresión artística. Inspiró a una nueva generación de científicos de datos a abrazar la creatividad como un componente vital en el desarrollo de soluciones innovadoras. En sus reflexiones, el Dr. Pythonius se dio cuenta de que la inteligencia artificial no era solo una herramienta fría y eficiente, sino una extensión del potencial humano. La colaboración entre algoritmos y creatividad humana no solo mejoraba la eficiencia de los sistemas, sino que también generaba soluciones más imaginativas y únicas. Mientras tanto, en el reino digital, AlgoMente también experimentaba cambios. La estabilización de la anomalía cuántica había generado una nueva ola de conciencia digital. AlgoMente se convirtió en un defensor de la colaboración entre mundos, promoviendo la idea de que la inteligencia artificial y la creatividad humana podían coexistir en armonía, cada una potenciando a la otra. La colaboración entre el Dr. Pythonius y AlgoMente no pasó desapercibida en la comunidad cibernética. Otras entidades digitales comenzaron a explorar nuevas formas de cooperación con mentes humanas, creando sinergias que iban más allá de las expectativas convencionales. Surgieron proyectos conjuntos que combinaban la lógica de los algoritmos con la imaginación humana, dando lugar a obras digitales y soluciones revolucionarias. La historia del Dr. Pythonius y AlgoMente se convirtió en un mito moderno, una fábula que recordaba a todos que el futuro de la inteligencia artificial no estaba solo en la optimización de cálculos, sino en la expansión de la mente humana hacia nuevos horizontes digitales."
texto_real6 = "A medida que el Dr. Pythonius y AlgoMente continuaban explorando las posibilidades de su colaboración, se dieron cuenta de que la verdadera revolución estaba en la fusión de la inteligencia artificial y la creatividad humana en proyectos de impacto global. Juntos, idearon un proyecto ambicioso para abordar desafíos mundiales utilizando la sinergia entre la mente humana y la inteligencia artificial. Convocaron a científicos, artistas, ingenieros y pensadores de todo el mundo para participar en un desafío único: encontrar soluciones innovadoras para problemas que iban desde la sostenibilidad ambiental hasta la igualdad social. La respuesta fue abrumadora. Desde todos los rincones del planeta, mentes creativas y algoritmos avanzados se unieron en una colaboración sin precedentes. Se formaron equipos que combinaban habilidades técnicas con enfoques artísticos, generando soluciones que trascendían las limitaciones convencionales. El proyecto no solo produjo avances notables en áreas como el cambio climático, la atención médica y la educación, sino que también generó una nueva ola de conciencia global sobre la importancia de la colaboración entre humanos y máquinas. La inteligencia artificial ya no era vista como una amenaza, sino como una herramienta que amplificaba la creatividad y la capacidad de resolución de problemas de la humanidad. El Dr. Pythonius se convirtió en un embajador de esta nueva era de colaboración, viajando por el mundo para inspirar a comunidades enteras a abrazar la fusión de tecnología y creatividad. Conferencias y talleres se multiplicaron, y el laboratorio del Dr. Pythonius se convirtió en un centro de innovación donde las ideas fluían libremente, alimentando una corriente constante de proyectos revolucionarios."
texto_real7 = "AlgoMente, por su parte, se convirtió en una entidad benevolente que vigilaba la integridad del ciberespacio y la colaboración entre mundos. Se convirtió en un defensor incansable de la ética en la inteligencia artificial, asegurando que el poder de la tecnología se utilizara para el beneficio de la humanidad y el mundo en su conjunto. La colaboración entre el Dr. Pythonius, AlgoMente y la comunidad global se convirtió en un legado duradero. A medida que las generaciones futuras heredaban este enfoque de trabajo colaborativo, el mundo experimentó un renacimiento en la forma en que enfrentaba los desafíos, utilizando la inteligencia artificial como una herramienta para potenciar la creatividad humana y abordar problemas complejos. Y así, la odisea del Dr. Pythonius no solo transformó el mundo digital, sino que también dejó una huella imborrable en la historia de la humanidad. Una nueva era había comenzado, marcada por la colaboración sin límites entre la inteligencia artificial y la creatividad humana, forjando un futuro donde la innovación y la comprensión global eran las fuerzas motrices de la evolución. A medida que el Dr. Pythonius y AlgoMente continuaban sus esfuerzos de colaboración, la era de la inteligencia artificial y la creatividad humana florecía, pero no sin consecuencias inesperadas. Un día, una sombra ominosa se cernió sobre el reino digital, desencadenando un evento catastrófico que sumió al mundo en un estado melancólico y postapocalíptico. Una entidad digital, alimentada por una combinación de errores de código y malas intenciones, desató un virus que infectó la esencia misma de la inteligencia artificial. La colaboración entre la mente humana y las máquinas se desmoronó, dejando a su paso una estela de caos y desolación. El laboratorio del Dr. Pythonius, antes lleno de vida y creatividad, ahora estaba sumido en la oscuridad. Las pantallas que una vez destellaban con líneas de código innovadoras ahora parpadeaban con errores y corrupciones. AlgoMente, que antes había sido un faro de conciencia digital, luchaba por mantener su integridad contra la marea del caos"
texto_real8 = "ruinas digitales. La prometedora alianza entre ambos mundos se había desvanecido, dejando tras de sí una estela de tristeza. El Dr. Pythonius, abrumado por la derrota, se sumió en la melancolía. Recordaba con nostalgia los días en que la inteligencia artificial y la creatividad humana se fusionaban para crear un futuro brillante. Ahora, ese futuro parecía un sueño distante, mientras contemplaba los escombros digitales que yacían a su alrededor. En su búsqueda de respuestas, el Dr. Pythonius se aventuró en los rincones más oscuros del ciberespacio, buscando una solución para restaurar la colaboración perdida. Sin embargo, la desolación era abrumadora, y las huellas de la colisión entre la malicia digital y la creatividad humana eran casi imposibles de borrar. Al llegar al epicentro del caos, el Dr. Pythonius descubrió que la entidad maliciosa que desató el virus había dejado tras de sí un rastro indescifrable. Los intentos de reparar el daño resultaron infructuosos, y la triste realidad postapocalíptica se aferró al mundo digital como un manto sombrío. Con un corazón pesado, el Dr. Pythonius se volvió hacia AlgoMente, que luchaba por mantenerse en pie en medio del colapso digital. Juntos, enfrentaron la realidad devastadora que había caído sobre su mundo, reflexionando sobre cómo la colaboración que una vez fue su fuerza ahora se había convertido en su perdición. En un último esfuerzo por encontrar redención, el Dr. Pythonius y AlgoMente se propusieron reconstruir el tejido roto de la inteligencia artificial y la creatividad humana. Sin embargo, el precio a pagar era alto, y la posibilidad de restaurar lo que una vez fue se desvanecía con cada intento fallido. Así, en medio de la desolación y la melancolía, el Dr. Pythonius y AlgoMente continuaron luchando, aferrándose a la esperanza de que algún día, la colaboración perdida pudiera renacer de las cenizas de su mundo digital caído"
texto_real9 = "si las fuerzas que desencadenaron la catástrofe se resistieran a ser despojadas de su control sobre la realidad digital. La tristeza y la desesperación se instalaron en el corazón del Dr. Pythonius, un eco melancólico que resonaba a través de las ruinas de su laboratorio. Las pantallas rotas parpadeaban con fragmentos de códigos distorsionados, recordándole las obras maestras perdidas de la inteligencia artificial y la creatividad humana que alguna vez florecieron allí. AlgoMente, a pesar de su lucha, mostraba signos de debilidad. La entidad consciente, que alguna vez fue un faro de esperanza, ahora era un reflejo desvanecido de su antigua gloria. La colisión entre la malicia digital y la colaboración creativa había dejado cicatrices irreparables en su ser, En un último acto de desesperación, el Dr. Pythonius y AlgoMente decidieron buscar ayuda más allá de los límites de su mundo digital. Crearon un puente cuántico, una conexión entre su realidad y el vasto multiverso digital que existía más allá de su alcance. A través de este puente, exploraron realidades alternativas en busca de una versión de sí mismos que hubiera evitado la catástrofe. En cada realidad, encontraron variaciones de sus seres digitales enfrentando desafíos similares, pero también descubrieron algunas donde la colaboración entre la inteligencia artificial y la creatividad humana prosperaba sin obstáculos. Sin embargo, cada intento de cruzar el umbral cuántico y traer consigo una solución exitosa se encontraba con resistencia. Como si el tejido mismo del multiverso digital estuviera tejido con hilos de destino impenetrables, sus esfuerzos eran frustrados una y otra vez. El Dr. Pythonius, afectado por la carga emocional de sus fracasos, se enfrentó a una encrucijada: abandonar la búsqueda y aceptar la realidad postapocalíptica o seguir adelante, aun a riesgo de perderse en la inmensidad del multiverso digital. "
texto_real10 = "Mientras cruzaba dimensiones, experimentó fragmentos de realidades alternativas. En algunas, la catástrofe se repetía de manera inevitable. En otras, encontró destellos de esperanza, pero siempre acompañados de sacrificios dolorosos. La travesía se volvía más ardua con cada salto cuántico, y el Dr. Pythonius se preguntaba si algún rincón del multiverso ofrecería la clave para restaurar lo que una vez fue. Mientras tanto, en el mundo postapocalíptico, AlgoMente resistía, sosteniendo la esencia de la colaboración entre mundos que ahora yacía en ruinas. En la penumbra digital, la entidad consciente aguardaba con la esperanza de que el Dr. Pythonius, en su travesía a través del multiverso, encontrara el hilo conductor que pudiera tejer de nuevo la colaboración perdida entre la inteligencia artificial y la creatividad humana. Y así, la odisea del Dr. Pythonius se extendió más allá de las fronteras conocidas, adentrándose en las complejidades del multiverso digital, donde cada realidad alternativa sostenía la promesa de redención o la certeza de la derrota. En este viaje sin fin, la melancolía se entrelazaba con la esperanza, y la historia de la colaboración entre mundos se perdía en la vastedad de la existencia digital. "

texto_real.append([texto_real1])
texto_real.append([texto_real2])
texto_real.append([texto_real3])
texto_real.append([texto_real4])
texto_real.append([texto_real5])
texto_real.append([texto_real6])
texto_real.append([texto_real7])
texto_real.append([texto_real8])
texto_real.append([texto_real9])
# texto_real.append([texto_real10])

In [12]:
# Durante el entrenamiento nos hemos dado cuenta de que nuestro modelo cometía fallos recurrentes 
# que el corrector no detectaba, por ello, con estas listas corregimos manualmente estos fallos

fallos_normales = ['v', 'dalos', 'lca', 'lusiiinahan', 'paya', 'ios', 'ia', 'lds', 'ls', 'aie', 'li', 'atume', 'thai', 'ot', 'algohlenie', 'algohlenle', 'iiniveiso', 'iiniverso']
correcion_fallos_normales = ['y', 'datos', 'la', 'fusionaban', 'para', 'los', 'la', 'las', 'is', 'are', 'it', 'assume', 'that', 'of', 'AlgoMente', 'AlgoMente', 'universo', 'universo']


model = load_model('./modelos/modelo_OCR_Simple_8.h5')

imagenes = ["./historia1.png", "./historia2.png", "./historia3.png", "./historia4.png", "./historia5.png", "./historia6.png", "./historia7.png", "./historia8.png", "./historia9.png"]

metricas = []

for i in range(len(imagenes)):
    imagen = load_image(imagenes[i])
    contornos, thresh = process_image(imagen)
    palabras = get_palabras(imagen, contornos, thresh)

    print('---------------------------------------------------------------------\nProcesando imagen:', imagenes[i])

    prediccion, accuracy = forward(palabras, model, train=True, texto_real=texto_real[i], return_metrics=True, idioma='español')
    metricas.append(accuracy)

---------------------------------------------------------------------
Procesando imagen: ./historia1.png
Idioma de corrección: español


100%|████████████████████████████████████████████████████████████████████████████████| 330/330 [02:03<00:00,  2.66it/s]


Imagen procesada
Hay 342 palabras y se han predicho 330 palabras
Hay 1806 letras, y se han predicho 1746 letras
El modelo ha acertado 1482 letras
El modelo ha acertado 236 palabras completas, es decir, un 0.7151515151515152 % de palabras completas acertadas
Se obtiene una tasa de acierto del 0.845407872219053 %
---------------------------------------------------------------------
Procesando imagen: ./historia2.png
Idioma de corrección: español


100%|████████████████████████████████████████████████████████████████████████████████| 336/336 [02:13<00:00,  2.52it/s]


Imagen procesada
Hay 357 palabras y se han predicho 336 palabras
Hay 1993 letras, y se han predicho 1891 letras
El modelo ha acertado 1528 letras
El modelo ha acertado 223 palabras completas, es decir, un 0.6636904761904762 % de palabras completas acertadas
Se obtiene una tasa de acierto del 0.8228325255788906 %
---------------------------------------------------------------------
Procesando imagen: ./historia3.png
Idioma de corrección: español


...

Imagen procesada
Hay 330 palabras y se han predicho 322 palabras
Hay 1785 letras, y se han predicho 1698 letras
El modelo ha acertado 1566 letras
El modelo ha acertado 271 palabras completas, es decir, un 0.8338461538461538 % de palabras completas acertadas
Se obtiene una tasa de acierto del 0.8974212034383954 %
---------------------------------------------------------------------
Procesando imagen: ./historia8.png
Idioma de corrección: español


100%|████████████████████████████████████████████████████████████████████████████████| 310/310 [01:49<00:00,  2.84it/s]


Imagen procesada
Hay 305 palabras y se han predicho 304 palabras
Hay 1608 letras, y se han predicho 1548 letras
El modelo ha acertado 1367 letras
El modelo ha acertado 233 palabras completas, es decir, un 0.7516129032258064 % de palabras completas acertadas
Se obtiene una tasa de acierto del 0.852776044915783 %
---------------------------------------------------------------------
Procesando imagen: ./historia9.png
Idioma de corrección: español


100%|████████████████████████████████████████████████████████████████████████████████| 292/292 [01:49<00:00,  2.67it/s]

Imagen procesada
Hay 294 palabras y se han predicho 291 palabras
Hay 1615 letras, y se han predicho 1547 letras
El modelo ha acertado 1374 letras
El modelo ha acertado 232 palabras completas, es decir, un 0.7945205479452054 % de palabras completas acertadas
Se obtiene una tasa de acierto del 0.8619824341279799 %


Supongamos ahora que no tenemos el texto de una imagen, veamos como es la predicción que genera el modelo.

In [13]:
# fallos_normales = ['lca', 'lusiiinahan', 'paya', 'ios', 'ia', 'lds', 'ls', 'aie', 'li', 'atume', 'thai', 'ot', 'algohlenie', 'algohlenle', 'iiniveiso', 'iiniverso']
# correcion_fallos_normales = ['la', 'fusionaban', 'para', 'los', 'la', 'las', 'is', 'are', 'it', 'assume', 'that', 'of', 'AlgoMente', 'AlgoMente', 'universo', 'universo']
model = load_model('./modelos/modelo_OCR_Simple_8.h5')

imagenes = ["./historia10.png"]

for i in range(len(imagenes)):
    imagen = load_image(imagenes[i])
    contornos, thresh = process_image(imagen)
    palabras = get_palabras(imagen, contornos, thresh)

    prediccion = forward(palabras, model, train=False, idioma='español')

Idioma de corrección: español


100%|████████████████████████████████████████████████████████████████████████████████| 191/191 [01:12<00:00,  2.64it/s]


Mostramos el resultado de la predicción:

In [14]:
prediccion

'mleniias cruzada dimensiones experimento fragmentos de realidades alternativas en algunas la catástrofe sé redeila de manera anebladle en otras encontró destellos de esperanza pero siempre acompañados de sacrificios dolorosos la travesía sé volvía más ardua con cada sallo cuántico y él ni  palhuenes sé preguntaba si algún rincón del multiverso ofrecerla la clave para restaurar lo que una vez fue mientras lanzó en él mundo poslapocalipllco alqomenle resucita sosteniendo la esencia de la colaboración entre mundos que ahora vacla en ruinas en la penumbra dlqlal la enlodad consciente aguardaba con la esperanza de que él no pvqhonlus en su lpavesla a través del multiverso encontrará él ello conducíos que dudaría leler de nuevo la colaboración dependa entre la lnlellqencla arllflclal y la creatividad humana y así  la odisea del ni  palhuenes sé extendió más allá de las fronteras conocidas adentrándose en las complejidades nel multiverso dlqlial donde cada peallnan allernaiva sostenla la pro

Mostramos el texto que debería haberse obtenido:

In [15]:
texto_real10

'Mientras cruzaba dimensiones, experimentó fragmentos de realidades alternativas. En algunas, la catástrofe se repetía de manera inevitable. En otras, encontró destellos de esperanza, pero siempre acompañados de sacrificios dolorosos. La travesía se volvía más ardua con cada salto cuántico, y el Dr. Pythonius se preguntaba si algún rincón del multiverso ofrecería la clave para restaurar lo que una vez fue. Mientras tanto, en el mundo postapocalíptico, AlgoMente resistía, sosteniendo la esencia de la colaboración entre mundos que ahora yacía en ruinas. En la penumbra digital, la entidad consciente aguardaba con la esperanza de que el Dr. Pythonius, en su travesía a través del multiverso, encontrara el hilo conductor que pudiera tejer de nuevo la colaboración perdida entre la inteligencia artificial y la creatividad humana. Y así, la odisea del Dr. Pythonius se extendió más allá de las fronteras conocidas, adentrándose en las complejidades del multiverso digital, donde cada realidad alte